In [3]:
import streamlit as st
from streamlit_autorefresh import st_autorefresh
from datetime import datetime as dt
import pandas as pd
import psycopg2
import altair as alt
import os
from dotenv import load_dotenv

load_dotenv()

user = os.getenv('HAA_DB_USER')
password = os.getenv('HAA_DB_PASSWORD')
host = os.getenv('HAA_DB_HOST')


def make_connection():
    conn = psycopg2.connect(user=user, password=password, host=host, database='postgres', port=5432)
    return conn

def get_temperature_total():
    conn = make_connection()    
    sql = "select * from temperatuur where tijd > '"+str(dt.today())[0:10]+" 00:00:00';"
    df = pd.read_sql_query(sql, con = conn)[['tijd', 'buiten', 'keuken', 'kamer']].set_index('tijd')
    return df

# temperaturen vandaag
def get_temperature(ruimte):
    conn = make_connection()
    sql = "select * from temperatuur where tijd > '"+str(dt.today())[0:10]+" 00:00:00';"
    df = pd.read_sql_query(sql, con = conn)[['tijd', ruimte]].set_index('tijd')
    df['uur'] = df.index.floor('H')
    df = df.set_index('uur')
    df = df.groupby('uur').mean()
    df['tijd'] = df.index

    return df

buiten = get_temperature('buiten')
kamer = get_temperature('kamer')
keuken = get_temperature('keuken')
alles = get_temperature_total()

alles = buiten.buiten.to_list()+keuken.keuken.to_list()+kamer.kamer.to_list()
min_value = int(min(alles))
max_value = round(max(alles), 0)



/home/kali/git/kitchen/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kali/git/kitchen/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kali/git/kitchen/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/kali/git/kitchen/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(

In [10]:
buiten.head()

,buiten
uur,
2022-04-11 00:00:00,2.512200
2022-04-11 01:00:00,2.155833
2022-04-11 02:00:00,1.749833
2022-04-11 03:00:00,1.416667
2022-04-11 04:00:00,1.166667


In [27]:
max(buiten.buiten)
min(buiten.buiten)

1.1666666666666667

In [2]:
## Plots normaliseren

def get_min_max():


In [23]:
alt.renderers.enable('mimetype')
alt.Chart(buiten).mark_line().encode(
    x=alt.X('tijd'),
    y=alt.Y('buiten', scale=alt.Scale(domain=[min_value, max_value]))
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


alt.Chart(df).l